In [1]:
#!pip install transformers datasets accelerate huggingface_hub

In [2]:
# !pip install bitsandbytes

In [3]:
import os
import json
from getpass import getpass
from huggingface_hub import login

In [4]:
token = getpass(prompt="enter huggingface token")
login(token=token)

enter huggingface token ········


In [5]:
dir_name = "data"

if os.path.isfile(dir_name + '/topics.json'):
    with open(dir_name + '/topics.json') as t:
        topics = json.load(t)

    topics = topics.keys()
    tfiles = [dir_name + f"""/{"-".join(t.lower().split(' '))}.json""" for t in topics]
    tfiles = [tf for tf in tfiles if os.path.isfile(tf)]

    data = []
    for tf in tfiles:
        with open(tf) as t:
            data.extend(json.load(t))

    print(f'there are {len(data)} rows in the dataset')

there are 180 rows in the dataset


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

In [7]:
dataset = load_dataset("json", data_files=tfiles)

In [8]:
dataset = dataset["train"].train_test_split(test_size=0.3, seed=42)

In [9]:
def preprocess(row):
    output_texts = []
    for i in range(len(row['question'])):
        text = f'<question>\n{row["question"][i]}\n</question>\n\n<documents>\n{row["documents"][i]}\n</documents>\n\n<answer>\n{row["answer"][i]}\n</answer>'
        output_texts.append(text)
    return output_texts

In [10]:
# dataset = dataset.map(preprocess).shuffle(seed=42)

In [11]:
# print(dataset['train'][0]['input'])

In [20]:
from transformers import BitsAndBytesConfig

model_id = "meta-llama/Llama-3.2-1B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [21]:
# model = model.to(torch.device('cpu'))

In [22]:
tokenizer.pad_token = tokenizer.eos_token

In [23]:
# def tokenize_function(row):
#     prompt = row["input"]
#     full_text = prompt + "\n" + f'{row["answer"]}\n</answer>'

#     tokenized = tokenizer(
#         full_text,
#         truncation=True,
#         max_length=1536,
#         padding="max_length",
#     )

#     prompt_tokenized = tokenizer(
#         prompt,
#         truncation=True,
#         max_length=1536,
#         add_special_tokens=False  # Ensure no extra tokens are added
#     )
#     prompt_length = len(prompt_tokenized["input_ids"])

#     # Copy the tokenized full text to create labels and mask out prompt tokens
#     labels = tokenized["input_ids"].copy()
#     labels[:prompt_length] = [-100] * prompt_length  # -100 tells PyTorch to ignore these tokens in loss computation

#     tokenized["labels"] = labels
#     return tokenized

In [24]:
# tokenized_dataset = dataset.map(tokenize_function)

In [25]:
from peft import LoraConfig, get_peft_model

# Define LoRA Configuration
lora_config = LoraConfig(
    r=4,               # Low-rank dimension
    lora_alpha=32,     # Scaling factor
    lora_dropout=0.1,  # Dropout for regularization
    bias="none",       # No bias training
    task_type="CAUSAL_LM"  # Task type: Causal Language Modeling
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 425,984 || all params: 1,236,240,384 || trainable%: 0.0345


In [26]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [27]:
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

response_template = "<answer>"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
    model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    args=SFTConfig(output_dir="/tmp"),
    formatting_func=preprocess,
    data_collator=collator,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/54 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/usr/local/lib/python3.11/site-packages/trl/trainer/utils.py:160: UserWarning: Could not find response key `<answer>` in the following instance: <|begin_of_text|><question>
How has coffee influenced literature, art, and intellectual movements?
</question>

<documents>
Document 1: Coffee houses in 17th and 18th century Europe served as crucial venues for the development and dissemination of Enlightenment ideas. London's coffee houses were nicknamed "penny universities" because the price of a cup granted access to newspapers, pamphlets, and vigorous intellectual debate. Notable figures like Isaac Newton, Samuel Pepys, and later, Samuel Johnson, were regular patrons. The first scientific paper published by the Royal Society was presented initially at a coffee house. In France, Café Procope became a ce

Step,Training Loss


/usr/local/lib/python3.11/site-packages/trl/trainer/utils.py:160: UserWarning: Could not find response key `<answer>` in the following instance: <|begin_of_text|><question>
How has 3D digital clothing design changed traditional fashion production processes?
</question>

<documents>
Document 1: Fashion weeks have expanded beyond traditional capitals to include regional showcase events highlighting local design talent. Cities like Copenhagen, Seoul, and Lagos have established distinct fashion weeks that attract international attention while emphasizing their unique aesthetic perspectives and designer communities. These events provide alternative platforms for emerging designers who may approach fashion with different cultural contexts and business models than those typically represented in established fashion capitals.
---
Document 2: The luxury resale market has developed sophisticated authentication procedures to address counterfeit concerns. Leading resale platforms combine technologi

KeyboardInterrupt: 

In [ ]:
#torch.cuda.empty_cache()

In [ ]:
# from transformers import pipeline

# pipe = pipeline("text-generation",
#                 model="./llama-1b-finetuned/checkpoint-224", 
#                 device="cuda", 
#                 max_length=1536,
#                 truncation=True)

In [ ]:
# response = pipe(
#     """
# <question>
# How should we treat animals?
# </question>

# <documents>
# Document 1: But it’s been thirty years and vegans are still less than 10% of the population.If you genuinely care about animal suffering, you have to admit that, and say, ‘what else can we do to ease animal suffering?’”After Singer’s talk, I began thinking through the consequences of his morality.A question occurred to me: “Should we also stop animals from eating each other?” I was sure others had made such arguments as *reductio ad absurdums* of vegetarianism, but I thought I might be the first to be genuinely interested in it from a moral perspective.
# ---
# Document 2: They didn’t even pretend to make an argument; it was entirely mockery.Watching this, I couldn’t help but realize there was a powerful logical argument at the core of the animal rights groups: animals should be treated much the same way humans are — their lives should be respected, their pain minimized, etc.Make this one simple change to your system of morality and everything else falls into place.
# ---
# Document 3: But I praise people for the good things they do and condemn them for the bad ones.”A final question raised the incrementalism versus revolutionism debate common to all left-wing social movements.Should one really worry about animal treatment when the animals were still going to be killed?Pinger said the answer was undoubtedly yes.
# ---
# Document 4: They didn’t even pretend to make an argument; it was entirely mockery.Watching this, I couldn’t help but realize there was a powerful logical argument at the core of the animal rights groups: animals should be treated much the same way humans are — their lives should be respected, their pain minimized, etc.Make this one simple change to your system of morality and everything else falls into place.
# ---
# Document 5: But I praise people for the good things they do and condemn them for the bad ones.”A final question raised the incrementalism versus revolutionism debate common to all left-wing social movements.Should one really worry about animal treatment when the animals were still going to be killed?Pinger said the answer was undoubtedly yes.
# ---
# Document 6: But it’s been thirty years and vegans are still less than 10% of the population.If you genuinely care about animal suffering, you have to admit that, and say, ‘what else can we do to ease animal suffering?’”After Singer’s talk, I began thinking through the consequences of his morality.A question occurred to me: “Should we also stop animals from eating each other?” I was sure others had made such arguments as *reductio ad absurdums* of vegetarianism, but I thought I might be the first to be genuinely interested in it from a moral perspective.
# </documents>

# <answer>
# """.strip()
# )
# print(response[0]['generated_text'])